In [ ]:
# !pip install fitz

In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install peft
# !pip install accelerate
# !pip install bitsandbytes
# !pip install
# !pip install torch

In [ ]:
!pip install pymupdf
import fitz  # Import the fitz module from PyMuPDF

#Extract text from the given PDF file
def extract_text_from_pdf(pdf_path):
  pdf_path ="/content/Nlp and llm content.pdf"
  doc = fitz.open(pdf_path) # Now fitz is defined and can be used.
  text = "\n".join([page.get_text("text") for page in doc])
  return text

In [ ]:
# Add this import statement at the beginning of your script (ipython-input-22-d8fb31f8fa31)
from datasets import Dataset

def create_dataset_from_text(pdf_path):
    raw_text = extract_text_from_pdf(pdf_path)
    text_chunks = raw_text.split("\n\n")  # Split text into manageable chunks

    instructions, responses = [], []
    for i in range(0, len(text_chunks) - 1, 2):  # Assume alternating Q/A format
        instructions.append(text_chunks[i].strip())
        responses.append(text_chunks[i + 1].strip())

    dataset = Dataset.from_dict({"instruction": instructions, "output": responses}) # Now Dataset is defined and can be used.
    return dataset

dataset_path = "/content/Nlp and llm content.pdf"
dataset = create_dataset_from_text(dataset_path)

In [ ]:
# ✅ Load the Model and Tokenizer with CPU offloading
from transformers import AutoTokenizer, AutoModelForCausalLM # Importing AutoTokenizer and AutoModelForCausalLM
import torch

def load_model_and_tokenizer(model_name="unsloth/Phi-4"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        load_in_4bit=True,  # Enable quantization for efficiency
        llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offloading if necessary
    )
    return model, tokenizer

model, tokenizer = load_model_and_tokenizer()

In [ ]:
#✅ Fine-tune the model with LoRA
from peft import LoraConfig, get_peft_model # Import LoraConfig and get_peft_model

def fine_tune_model(model, dataset, lora_rank=8):
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
    )

    model = get_peft_model(model, lora_config)

    # Enable Gradient Checkpointing
    model.enable_input_require_grads()
    model.gradient_checkpointing_enable()

    return model

model = fine_tune_model(model, dataset)

In [ ]:
# ✅ Generate response using the fine-tuned model
def generate_response(prompt, model, tokenizer, max_length=256):
    input_ids = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        add_special_tokens=True
    ).input_ids.to(model.device)

    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.9,
        top_k=40,
        top_p=0.85,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# ✅ Test the fine-tuned model
import datetime
import time

test_prompt = "What is NLP and how is it categorized"

# Record start time
start_time = time.time()

response = generate_response(test_prompt, model, tokenizer)

# Record end time and calculate execution time
end_time = time.time()
execution_time = end_time - start_time

# Print current time, execution time, and response
print(f"Current Time: {datetime.datetime.now()}")
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Response: {response}")